# Detekcja krawędzi

## Cel ćwiczenia

- Zapoznanie z metodami detekcji krawędzi:
    - Sobel, Prewitt, Roberts - przypomnienie,
    - Laplasjan z Gaussa (LoG – ang. Laplacian of Gaussian),
    - Canny.

Detekcja krawędzi przez wiele lat była podstawą algorytmów segmentacji.
Krawędzie wykrywane są najczęściej z wykorzystaniem pierwszej (gradient) i drugiej (Laplasjan) pochodnej przestrzennej.
Wykorzystanie obu metod zaprezentowane zostało w ćwiczeniu *Przetwarzanie wstępne. Filtracja kontekstowa*.

W niniejszym ćwiczeniu poznane detektory krawędzi zostaną porównane z bardziej zaawansowanymi: Laplasjan z funkcji Gaussa (LoG), Zero Crossing i Canny.

## Laplasjan z Gaussa (LoG)

Funkcja Gaussa:<br>
\begin{equation}
h(r) = e^{\frac{-r^2}{2 \sigma^2}}
\end{equation}<br>
gdzie:
- $r^2 = x^2 + y^2$
- $\sigma$ to odchylenie standardowe.

Działanie filtracji Gaussowskiej zostało przedstawione w ćwiczeniu "Przetwarzanie wstępne". W jej wyniku następuje rozmazanie obrazu.
Laplasjan tej funkcji dany jest wzorem:

\begin{equation}
\nabla^2 h(r) = \frac{r^2 - 2\sigma^2}{\sigma^4} e^{-\frac{r^2}{2\sigma^2}}
\end{equation}

Funkcję (z oczywistych powodów) nazywamy Laplasjan z Gaussa (LoG).
Ponieważ druga pochodna jest operacją liniową, konwolucja obrazu z $\nabla^2 h(r)$ daje taki sam efekt jak zastosowanie filtracji Gaussa na obrazie, a następnie obliczenie Laplasjanu z wyniku.
Lokalizacja krawędzi polega na znalezieniu miejsca, gdzie po filtracji LoG następuje zmiana znaku.

1. Wczytaj obraz *house.png*.
2. Wykonaj rozmycie Gaussowskie obrazu wejściowego.
W tym celu wykorzysaj funkcję `cv2.GaussianBlur(img, kSize, sigma)`.
Pierwszy argument jest obrazem wejśćiowym.
Drugi jest rozmiarem filtru (podanym w nawiasach okrągłych, np. *(3, 3)*).
Trzecim argumentem jest odchylenie standardowe. Wartość jest dobrana automatycznie, jeśli zosanie podana wartość `0` (będą równe rozmiarowi).
3. Oblicz laplasjan obrazu rozmytego.
W tym celu wykorzysaj funkcję `cv2.Laplacian(img, ddepth)`.
Pierszym argumentem jest obraz wejściowy.
Drugim argumentem jest typ danych wejściowych. Użyj `cv2.CV_32F`.
4. Wyznacz miejsca zmiany znaku.
Zaimplementuj funkcję `crossing(LoG, thr)`:
    - Najpierw stwórz tablicę, do której zostanie zapisany wynik.
    Jej rozmiar jest taki sam jak przetwarzanego obrazu.
    - Następnie wykonaj pętle po obrazie (bez ramki jednopikselowej).
    W każdej iteracji stwórz otoczenie o rozmiarze $3 \times 3$.
    Dla otoczenia oblicz wartość maksymalną i minimalną.
    - Jeśli wartości te mają przeciwne znaki, to do danego miejsca tablicy przypisz wartość:
        - jeśli piksel wejściowy > 0, to dodaj do niego wartość bezwzględną minimum.
        - jeśli piksel wejściowy < 0, to do jego wartości bezwzględnej dodaj maksimum.
    - Zmień zakres wykonanej tablicy do $<0, 255>$.
    - Wykonaj progowanie tablicy. Próg jest argumentem wejściowym.
    - Przeskaluj dane binarne do wartości `[0, 255]`.
    - Wykonaj konwersję do typu *uint8*.
    - Wykonaj rozmycie medianowe wyniku.
    Wykorzystaj funkcję `cv2.medianBlur(img, kSize)`.
    Pierwszym argumentem jest obraz wejśćiowy, a drugim rozmiar filtra.
    - Zwróć wyznaczoną tablicę.
5. Wyświetl obraz wynikowy.
6. Dobierz parametry (rozmiar filtru Gaussa, odchylenie standardowe, próg binaryzacji) tak, by widoczne były kontury domu, ale nie dachówki.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os
import requests

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/'

fileNames = ["dom.png"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)



In [ ]:
def crossing(LoG, thr):
  ret_mat = np.zeros(LoG.shape)

  for i in range(1, LoG.shape[0]-1):
    for j in range(1, LoG.shape[1]-1):
      ret_mat[i, j] = LoG[i, j]

      window = LoG[i-1:i+2, j-1:j+2]
      max_val = np.max(window)
      min_val = np.min(window)
      if(max_val * min_val < 0):
        if(LoG[i, j] > 0):
          ret_mat[i,j] = np.abs(min_val)
        elif(LoG[i,j] < 0):
          ret_mat[i,j] = np.abs(max_val)

  ret_mat = ret_mat.astype('uint8')
  ret_mat = cv2.threshold(ret_mat, thr, 255, cv2.THRESH_BINARY)
  ret_mat = cv2.medianBlur(ret_mat[1], 3)
  return ret_mat


In [ ]:
def LoG(I, thr):
  gauss = cv2.GaussianBlur(I, (3,3), 0)
  lapl = cv2.Laplacian(gauss, cv2.CV_32F)
  return crossing(lapl, thr)

In [ ]:
house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)

fig, axs = plt.subplots(1, 2)

axs[0].imshow(house, 'gray')
axs[1].imshow(LoG(house, 15), 'gray')
plt.show()


## Algorytm Canny'ego

> Algorytm Canny'ego to często wykorzystywana metoda detekcji krawędzi.
> Zaproponowana została w 1986r. przez Johna F. Cannego.
> Przy jego projektowaniu założono trzy cele:
> - niska liczba błędów - algorytm powinien znajdywać wszystkie krawędzie oraz generować jak najmniej fałszywych detekcji,
> - punkty krawędziowe powinny być poprawnie lokalizowane - wykryte punkty powinny być jak najbardziej zbliżone do rzeczywistych,
> - krawędzie o szerokości 1 piksela - algorytm powinien zwrócić jeden punkt dla każdej rzeczywistej krawędzi.

Zaimplementuj algorytm detekcji krawędziCanny'ego:
1. W pierwszym kroku obraz przefiltruj dwuwymiarowym filtrem Gaussa.
2. Następnie oblicz gradient pionowy i poziomy ($g_x $ i $g_y$).
Jedną z metod jest filtracja Sobela.
3. Dalej oblicz amplitudę:
$M(x,y)  = \sqrt{g_x^2+g_y^2}$ oraz kąt:
$\alpha(x,y) = arctan(\frac{g_y}{g_x})$.
Do obliczenia kąta wykorzystaj funkcję `np.arctan2(x1, x2)`.
Wynik jest w radianach.
4. W kolejnym etapie wykonaj kwantyzację kątów gradientu.
Kąty od $-180^\circ$ do $180^\circ$ można podzielić na 8 przedziałów:
[$-22.5^\circ, 22.5^\circ$], [$22.5^\circ, 67.5^\circ$],
[$67.5^\circ, 112.5^\circ$], [$112.5^\circ, 157.5^\circ$],
[$157.5^\circ, -157.5^\circ$], [$-157.5^\circ, -112.5^\circ$],
[$-112.5^\circ, -67.5^\circ$], [$-67.5^\circ, -22.5^\circ$].
Przy czym należy rozpatrywać tylko 4 kierunki:
    - pionowy ($d_1$),
    - poziomy ($d_2$),
    - skośny lewy ($d_3$),
    - skośny prawy ($d_4$).
5. Dalej przeprowadź eliminację pikseli, które nie mają wartości maksymalnej (ang. *nonmaximal suppresion*).
Celem tej operacji jest redukcja szerokości krawędzi do rozmiaru 1 piksela.
Algorytm przebiega następująco.
W rozpatrywanym otoczeniu o rozmiarze $3 \times 3$:
    - określ do którego przedziału należy kierunek gradientu piksela centralnego ($d_1, d_2, d_3, d_4$).
    - przeanalizuje sąsiadów leżących na tym kierunku.
Jeśli choć jeden z nich ma amplitudę większą niż piksel centralny, to należy uznać, że nie jest lokalnym maksimum i do wyniku przypisać $g_N(x,y) = 0$.
W przeciwnym przypadku $g_N(x,y) = M(x,y)$.
Przez $g_N$ rozumiemy obraz detekcji lokalnych maksimów.
Zaimplementuj funkcję `nonmax`.
Pierwszym argementem jest macierz kierunków (po kwantyzacji).
Drugim argumentem jest macierz amplitudy.
6. Ostatnią operacją jest binaryzacja obrazu $g_N$.
Stosuje się tutaj tzw. binaryzację z histerezą.
Wykorzystuje się w niej dwa progi: $T_L$ i $T_H$, przy czym $T_L < T_H$.
Canny zaproponował, aby stosunek progu wyższego do niższego był jak 3 lub 2 do 1.
Rezultaty binaryzacji można opisać jako:<br>
$g_{NH}(x,y) = g_N(x,y) \geq TH $<br>
$g_{NL}(x,y) = TH > g_N(x,y) \geq TL $<br>
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Dlatego wykorzystuje się obraz $g_{NL}$ w następującej procedurze:
    - Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
    - Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
    - Stwórz obraz, któy zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
    - Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź (potencjalną).
        Można to wykoanać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować wbardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj algorytm Cannego.

In [ ]:
def canny_edge_detection(image, low_threshold, high_threshold):
    # 1. Filtracja Gaussa
    blurred = cv2.GaussianBlur(image, (5, 5), 20)

    # 2. Obliczanie gradientów pionowych i poziomych (Sobel)
    sobelx = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=7)
    sobely = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=7)

    # 3. Obliczanie amplitudy i kątów gradientu
    magnitude = np.sqrt(sobelx**2 + sobely**2)
    angle = np.arctan2(sobely, sobelx) * (180 / np.pi)

    # 4. Kwantyzacja kątów gradientu
    quantized_angle = np.zeros_like(angle, dtype=np.uint8)
    quantized_angle[(angle >= -22.5) & (angle < 22.5)] = 0
    quantized_angle[(angle >= 22.5) & (angle < 67.5)] = 45
    quantized_angle[(angle >= 67.5) & (angle < 112.5)] = 90
    quantized_angle[(angle >= 112.5) & (angle < 157.5)] = 135
    quantized_angle[(angle >= 157.5) | (angle < -157.5)] = 0
    quantized_angle[(angle >= -157.5) & (angle < -112.5)] = 45
    quantized_angle[(angle >= -112.5) & (angle < -67.5)] = 90
    quantized_angle[(angle >= -67.5) & (angle < -22.5)] = 135

    # 5. Eliminacja nieistotnych pikseli (nonmaximal suppression)
    nonmax_suppressed = np.zeros_like(magnitude)
    rows, cols = magnitude.shape
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            direction = quantized_angle[i, j]
            if direction == 0:
                neighbors = [magnitude[i, j - 1], magnitude[i, j + 1]]
            elif direction == 45:
                neighbors = [magnitude[i - 1, j - 1], magnitude[i + 1, j + 1]]
            elif direction == 90:
                neighbors = [magnitude[i - 1, j], magnitude[i + 1, j]]
            elif direction == 135:
                neighbors = [magnitude[i - 1, j + 1], magnitude[i + 1, j - 1]]

            if magnitude[i, j] >= max(neighbors):
                nonmax_suppressed[i, j] = magnitude[i, j]

    # 6. Binaryzacja z histerezą
    edges = np.zeros_like(nonmax_suppressed)
    high_threshold_value = nonmax_suppressed.max() * high_threshold
    low_threshold_value = high_threshold_value * low_threshold

    strong_edges_i, strong_edges_j = np.where(nonmax_suppressed >= high_threshold_value)
    weak_edges_i, weak_edges_j = np.where((nonmax_suppressed >= low_threshold_value) & (nonmax_suppressed < high_threshold_value))

    edges[strong_edges_i, strong_edges_j] = 255
    edges[weak_edges_i, weak_edges_j] = 50  # 50 represents weak edges

    # 7. Procedura propagacji krawędzi
    stack = list(zip(strong_edges_i, strong_edges_j))
    visited = set(stack)
    while stack:
        current_i, current_j = stack.pop()
        for di in range(-1, 2):
            for dj in range(-1, 2):
                ni, nj = current_i + di, current_j + dj
                if (ni, nj) not in visited and 0 <= ni < rows and 0 <= nj < cols and edges[ni, nj] == 50:
                    edges[ni, nj] = 255
                    stack.append((ni, nj))
                    visited.add((ni, nj))

    return edges

In [ ]:
def canny(I, tl, th):
  gauss = cv2.GaussianBlur(I, (3,3), 0)


  gx = cv2.Sobel(gauss, cv2.CV_64F, 1, 0, ksize=7)
  gy = cv2.Sobel(gauss, cv2.CV_64F, 0, 1, ksize=7)

  #gx = gx.astype('float')
  #gy = gy.astype('float')

  M = np.sqrt(gx**2 + gy**2)
  alpha = np.arctan2(gy, gx) * (180 / np.pi)

  quantized_angle = np.zeros_like(alpha, dtype=np.uint8)
  quantized_angle[(alpha >= -22.5) & (alpha < 22.5)] = 2
  quantized_angle[(alpha >= 22.5) & (alpha < 67.5)] = 3
  quantized_angle[(alpha >= 67.5) & (alpha < 112.5)] = 1
  quantized_angle[(alpha >= 112.5) & (alpha < 157.5)] = 4
  quantized_angle[(alpha >= 157.5) | (alpha < -157.5)] = 2
  quantized_angle[(alpha >= -157.5) & (alpha < -112.5)] = 3
  quantized_angle[(alpha >= -112.5) & (alpha < -67.5)] = 1
  quantized_angle[(alpha >= -67.5) & (alpha < -22.5)] = 4

  gn = nonmax(quantized_angle, M)

  gnh = gn >= th
  gnl = (gn < th) & (gn >= tl)


  coord_list = []

  for i in range(0, gnh.shape[0]):
    for j in range(0, gnh.shape[1]):
      if gnh[i, j]:
        coord_list.append([i, j])


  rows, cols = M.shape

  edges = np.zeros_like(gn)
  high_threshold_value = gn.max() * th
  low_threshold_value = high_threshold_value * tl

  strong_edges_i, strong_edges_j = np.where(gn >= high_threshold_value)
  weak_edges_i, weak_edges_j = np.where((gn >= low_threshold_value) & (gn < high_threshold_value))

  edges[strong_edges_i, strong_edges_j] = 255
  edges[weak_edges_i, weak_edges_j] = 50  # 50 represents weak edges

    # 7. Procedura propagacji krawędzi
  stack = list(zip(strong_edges_i, strong_edges_j))
  visited = set(stack)
  while stack:
      current_i, current_j = stack.pop()
      for di in range(-1, 2):
          for dj in range(-1, 2):
              ni, nj = current_i + di, current_j + dj
              if (ni, nj) not in visited and 0 <= ni < rows and 0 <= nj < cols and edges[ni, nj] == 50:
                  edges[ni, nj] = 255
                  stack.append((ni, nj))
                  visited.add((ni, nj))


  #visited = np.zeros(I.shape)
  #edges = np.zeros(I.shape)

  #while len(coord_list) > 0:
  #  coord = coord_list.pop()
  #  if visited[coord[0], coord[1]] != 1:
  #    visited[coord[0], coord[1]] = 1
  #    edges[coord[0], coord[1]] = 1
  #    for ii in range(coord[0]-1, coord[0]+2):
  #      for jj in range(coord[1]-1, coord[1]+2):
  #        if gnl[ii, jj] == True:
  #          coord_list.append([ii, jj])

  return gnh, edges

def nonmax(angles, M):

  gn = np.zeros(M.shape)

  for i in range(1, M.shape[0]-1):
    for j in range(1, M.shape[1]-1):
      window = M[i-1:i+2, j-1:j+2]
      n_1 = window[1, 1]
      n_2 = window[1, 1]

      if angles[i, j] == 1: #Pionowy
        n_1 = window[0, 1]
        n_2 = window[2, 1]
      elif angles[i, j] == 2: #Poziomy
        n_1 = window[1, 0]
        n_2 = window[1, 2]
      elif angles[i, j] == 3: #Skosny lewy
        n_1 = window[0, 0]
        n_2 = window[2, 2]
      else: #Skosny prawy
        n_1 = window[2, 0]
        n_2 = window[0, 2]


      if n_1 <= window[1, 1] and n_2 <= window[1, 1]:
        gn[i, j] = window[1, 1]
  return gn

In [ ]:
house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)

gnh, edges = canny(house, 0.10, 0.15)
sol2 = canny_edge_detection(house, 0.10, 0.15)

fig, axs = plt.subplots(1, 4)
fig.set_size_inches(20, 5)

axs[0].imshow(house, 'gray')
axs[1].imshow(gnh, 'gray')
axs[2].imshow(edges, 'gray')
axs[3].imshow(kasia_edges, 'gray')
plt.show()

## Algorytm Canny'ego - OpenCV

1. Wykonaj dektekcję krawędzi metodą Canny'ego wykorzystując funkcję `cv2.Canny`.
    - Pierwszym argumentem funkcji jest obraz wejściowy.
    - Drugim argumentem jest mniejszy próg.
    - Trzecim argumentem jest większy próg.
    - Czwarty argument to tablica, do której wpisany zostanie wynik.
    Można zwrócić go przez wartość i podać wartość `None`.
    - Piąty argument to rozmiar operatora Sobela (w naszym przypadku 3).
    - Szósty argument to rodzaj używanej normy.
    0 oznacza normę $L_1$, 1 oznacza normę $L_2$. Użyj $L_2$.
2. Wynik wyświetl i porównaj z własną implementacją.

In [ ]:
house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)

canny_house = cv2.Canny(house, 80, 160, None, 3, 1)
gnh, edges = canny(house, .10, .15)


fig, axs = plt.subplots(1, 3)
fig.set_size_inches(15, 5)


axs[0].imshow(house, 'gray')
axs[1].imshow(canny_house, 'gray')
axs[2].imshow(edges, 'gray')
plt.show()